In [1]:
import sys
sys.path.append("../")
sys.path.append("../Modules/")

from Modules.simulation import Simulation
from Modules.cell_builder import SkeletonCell, CellBuilder
from Modules.constants import HayParameters

import os
from neuron import h

from logger import Logger # type: ignore

import numpy as np

--No graphics will be displayed.


In [2]:
%cd ../scripts

/home/drfrbc/Neural-Modeling/scripts


In [3]:
try:
    os.system("rm -r x86_64/")
except:
    pass

rm: cannot remove 'x86_64/': No such file or directory


## Define Simulations

In [4]:
import numpy as np

def generate_simulations(neuron_random_states, numpy_random_states, varying_attributes, common_attributes):
    # Generate the list of HayParameters with updated sim_names
    all_parameters = []
    
    # Define a helper function to create parameters
    def create_parameters(numpy_seed, neuron_seed, attributes, amp=None, excFR_increase=None):
        sim_name_parts = [attributes['base_sim_name'], f"Np{numpy_seed}"]
        if neuron_seed is not None:
            sim_name_parts.append(f"Neu{neuron_seed}")
        if amp is not None:
            sim_name_parts.append(f"amp{round(amp, 1)}")
        if excFR_increase is not None:
            sim_name_parts.append(f"EXCinc{round(excFR_increase, 1)}")
        sim_name = '_'.join(sim_name_parts)
        
        params = {
            **common_attributes,
            'numpy_random_state': numpy_seed,
            'sim_name': sim_name,
            **{k: v for k, v in attributes.items() if k != 'base_sim_name'}
        }
        
        if neuron_seed is not None:
            params['neuron_random_state'] = neuron_seed
        if amp is not None:
            params['h_i_amplitude'] = round(amp, 1)
        if excFR_increase is not None:
            params['excFR_increase'] = round(excFR_increase, 1)
            
        return HayParameters(**params)
    
    if not numpy_random_states:
        numpy_random_states = [None]
    if not neuron_random_states:
        neuron_random_states = [None]
    
    for numpy_seed in numpy_random_states:
        for neuron_seed in neuron_random_states:
            for attributes in varying_attributes:
                if 'CI_on' in common_attributes:
                    for amp in np.arange(0, 2.1, 0.5):
                        all_parameters.append(create_parameters(numpy_seed, neuron_seed, attributes, amp=amp))
                elif 'exc_constant_fr' in common_attributes:
                    for excFR_increase in np.arange(0, 8.1, 2):
                        all_parameters.append(create_parameters(numpy_seed, neuron_seed, attributes, excFR_increase=excFR_increase))
                else:
                    all_parameters.append(create_parameters(numpy_seed, neuron_seed, attributes))
    
    return all_parameters


synapse_keys = ['None']#, 'NoMapping'] #'MappingMerging'] #'Mapping', 'Merging', 
use_SA_probs = True
syn_numbers_to_use = 'Full'#'1000'#'Full' # TODO replace with keys
common_attributes_to_use = 'sta'#'sta'#'FI_ExcFR' # 'sta' , 'FI' , 'FI_ExcFR' for example
morphology_keys = ['Complex']#,'Branches', 'Trees']#['Complex']#['Complex', 'Branches', 'Trees']
replace_w_CI_keys = ['None']#, 'Tufts'] #'Basals&Tufts'
numpy_random_states = [5]#,10, 1000000]#, 5555555, 88888, 999999, 2222222, 7777777,66666,33333]
neuron_random_states = None

sim_title = 'TuningSynapses_reduceNA_shiftExcGmaxBy20Percent'#'FINoMapping'#'TuningFRSynapses'#'FSynapses'# if 'sta' in common_attributes_to_use else 'FI'

syn_numbers = {
    'Density': {'inh': None, 'exc': None},
    'Full': {'inh':2650, 'exc':26100},
    '1000': {'inh': int(1000 * (2650 / (26100 + 2650))), 'exc': int(1000 * (26100 / (26100 + 2650)))},
    '10000': {'inh': int(10000 * (2650 / (26100 + 2650))), 'exc':int(10000 * (26100 / (26100 + 2650)))}
}

# Define the template for common attributes
common_attributes = {
    'sta' : 
        {
    'h_tstop': 10000,#120000,
    'merge_synapses': False,
    'record_ecp': True,
    'record_all_channels':True,
    'record_all_synapses':True,
    'exc_use_density':True if syn_numbers_to_use == 'Density' else False,
    'inh_use_density':True if syn_numbers_to_use == 'Density' else False,
    'inh_syn_number': syn_numbers[syn_numbers_to_use]['inh'], # 150 soma syns already #2650
    'exc_syn_number': syn_numbers[syn_numbers_to_use]['exc'],  # 26100
    'use_SA_probs': use_SA_probs
    },
        
    'FI' : 
        {
    'h_tstop': 5000,
    'save_every_ms':5000,
    'all_synapses_off': False,
    'CI_on': True,
    'h_i_duration': 4950,
    'h_i_delay': 50,
    'exc_use_density': True if syn_numbers_to_use == 'Density' else False,
    'inh_use_density': True if syn_numbers_to_use == 'Density' else False,
    'inh_syn_number': syn_numbers[syn_numbers_to_use]['inh'], # 150 soma syns already #2650
    'exc_syn_number': syn_numbers[syn_numbers_to_use]['exc'],  # 26100
    'use_SA_probs': use_SA_probs
    },
    'FI_ExcFR' : # changes exc FR instead of changing CI amp
        {
    'h_tstop': 5000,
    'save_every_ms':5000,
    'all_synapses_off': False,
    'exc_constant_fr': True,
    'h_i_duration': 4950,
    'h_i_delay': 50,
    'exc_use_density': True if syn_numbers_to_use == 'Density' else False,
    'inh_use_density': True if syn_numbers_to_use == 'Density' else False,
    'inh_syn_number': syn_numbers[syn_numbers_to_use]['inh'], # 150 soma syns already #2650
    'exc_syn_number': syn_numbers[syn_numbers_to_use]['exc'],  # 26100
    'use_SA_probs': use_SA_probs
    }

}

morphology_attributes = {
    'Complex': {'base_sim_name': 'Complex'},
    'Branches': {'base_sim_name': 'Branches', 'reduce_obliques': True, 'reduce_tufts': True, 'reduce_basals': 3},
    'Trees': {'base_sim_name': 'Trees', 'reduce_apic': True, 'reduce_basals': 1}
}

replace_w_CI_attributes = {
    'None': {'sim_name_add_suffix': ''},
    'Basals': {'sim_name_add_suffix': 'REPBasals', 'num_basal_to_replace_with_CI':8},
    '1Basal': {'sim_name_add_suffix': 'REP1Basal', 'num_basal_to_replace_with_CI':1},
    'Tufts': {'sim_name_add_suffix': 'REPTufts', 'num_tuft_to_replace_with_CI':2},
    '1Tuft': {'sim_name_add_suffix': 'REP1Tuft', 'num_tuft_to_replace_with_CI':1},
    'Basals&Tufts': {'sim_name_add_suffix': 'REPBasals&Tufts', 'num_basal_to_replace_with_CI':8, 'num_tuft_to_replace_with_CI':2}
}

varying_syn_attributes = {
    'None': {'sim_name_add_suffix': ''},
    'NoMapping': {'sim_name_add_suffix': 'NoMapping', 'synapse_mapping': False},
    'Merging': {'sim_name_add_suffix': 'Merging', 'merge_synapses': True},
    'MappingMerging': {'sim_name_add_suffix': 'MappingMerging', 'synapse_mapping': True, 'merge_synapses': True}
}
mean = (np.log(0.45) - 0.5 * np.log((0.35/0.45)**2 + 1))
# Add synaptic gmax parameters
synaptic_gmax_params = {
    'inh_gmax_dist': 2,
    'soma_gmax_dist': 2,
    'exc_gmax_mean_0': mean - abs(0.2*mean),
    'exc_gmax_std_0': np.sqrt(np.log((0.35/0.45)**2 + 1)),
    'exc_gmax_clip': (0, 5),
    'exc_scalar': 1
}

conductance_params = {}

# Combine common attributes with synaptic gmax parameters
common_attributes = {**common_attributes[common_attributes_to_use], **synaptic_gmax_params}

# Generate varying attributes by combining morphology, replace_w_CI, and synapse attributes
varying_attributes = []
for morph_key in morphology_keys:
    if (morph_key == 'Complex') or (morph_key == 'Trees'): # Complex cell will not be having any dendrites replaced with Current injection
        replace_keys = ['None']
    else:
        replace_keys = replace_w_CI_keys
    for replace_key in replace_keys:
        for syn_key in synapse_keys:
            combined_attrs = {**morphology_attributes[morph_key], **replace_w_CI_attributes[replace_key], **varying_syn_attributes[syn_key]}
            combined_attrs['base_sim_name'] = f"{morphology_attributes[morph_key]['base_sim_name']}{replace_w_CI_attributes[replace_key].get('sim_name_add_suffix', '')}{varying_syn_attributes[syn_key].get('sim_name_add_suffix', '')}"
            combined_attrs.pop('sim_name_add_prefix', None)
            combined_attrs.pop('sim_name_add_suffix', None)
            varying_attributes.append(combined_attrs)

# Main execution code to generate simulations
all_parameters = generate_simulations(neuron_random_states, numpy_random_states, varying_attributes, common_attributes)


In [5]:
morphology_keys

['Complex']

In [6]:
all_parameters = generate_simulations(neuron_random_states, numpy_random_states, varying_attributes, common_attributes)

In [7]:
all_parameters

[HayParameters(sim_name='Complex_Np5', numpy_random_state=5, neuron_random_state=90, h_celcius=37, h_tstop=30000, h_dt=0.1, h_v_init=-77.2, CI_on=False, CI_target='soma', h_i_amplitude=10.0, h_i_duration=1000, h_i_delay=10, record_ecp=True, record_seg_to_seg=False, all_synapses_off=False, trunk_exc_synapses=True, perisomatic_exc_synapses=True, add_soma_inh_synapses=True, num_soma_inh_syns=450, inh_gmax_dist=2, soma_gmax_dist=2, exc_gmax_mean_0=-1.2420604129312118, exc_gmax_std_0=0.6878119625182042, exc_gmax_clip=(0, 5), exc_scalar=1, exc_synaptic_density=2.16, inh_synaptic_density=0.22, exc_use_density=False, inh_use_density=False, exc_syn_number=26100, inh_syn_number=2650, use_SA_probs=True, exc_P_release_mean=0.53, exc_P_release_std=0.22, inh_basal_P_release_mean=0.72, inh_basal_P_release_std=0.1, inh_apic_P_release_mean=0.3, inh_apic_P_release_std=0.08, inh_soma_P_release_mean=0.88, inh_soma_P_release_std=0.05, exc_syn_mod='AMPA_NMDA_STP', inh_syn_mod='GABA_AB_STP', inh_prox_mean_fr

## Run Cells

In [8]:
# try to run >64 (limit) simulation in batches
import math

# Define your batch size
batch_size = 64

# Check how many batches you will need
if len(all_parameters) > (batch_size - 1):
    number_of_batches = math.ceil(len(all_parameters) / batch_size)
    print(number_of_batches)
    
    # Create batches of indices
    batches = [all_parameters[i * batch_size:(i + 1) * batch_size] for i in range(number_of_batches)]
    
    # Run each batch
    for i, batch in enumerate(batches):
        sim = Simulation(SkeletonCell.Hay, title=sim_title)
        if i == 0:
            path_to_use = sim.path
        else:
            sim.path = path_to_use
        for parameters in batch:
            sim.submit_job(parameters)
        sim.run()
        
else:
    # Initialize simulation
    sim = Simulation(SkeletonCell.Hay, title=sim_title)

    # Submit jobs to simulation
    for parameters in all_parameters:
        # print(parameters)
        sim.submit_job(parameters)

    # Remove directory if it exists
    try:
        os.system("rm -r x86_64/")
    except:    pass

    # Run the simulation
    sim.run()


(2024-08-13 08:20:38.841692)-[PID: 1606183]–[INFO]: Total number of jobs: 1
(2024-08-13 08:20:38.841758)-[PID: 1606183]–[INFO]: Total number of proccessors: 192
(2024-08-13 08:20:38.841892)-[PID: 1606183]–[INFO]: Compiling modfiles.


rm: cannot remove 'x86_64/': No such file or directory


(2024-08-13 08:20:39.537723)-[PID: 1606495]–[INFO]: Building SkeletonCell.Hay.
(2024-08-13 08:20:39.816945)-[PID: 1606495]–[INFO]: CellModel: changed soma nseg from 5 to 1.
soma segments:[L5PCtemplate[0].soma[0](0.5)]
(2024-08-13 08:20:39.819923)-[PID: 1606495]–[INFO]: Building excitatory synapses.
Removing duplicate coordinate at index 1 in section L5PCtemplate[0].apic[0]
(2024-08-13 08:22:21.089589)-[PID: 1606495]–[INFO]: Building inhibitory synapses.
(2024-08-13 08:22:36.755218)-[PID: 1606495]–[INFO]: Building soma synapses.
(2024-08-13 08:22:36.795020)-[PID: 1606495]–[INFO]: Assigning excitatory spike trains.


In [ ]:
all_parameters

## Check Runtimes

In [ ]:
if common_attributes_to_use == 'sta':
    import analysis
    runtimes = {}

    try:
        for parameters in all_parameters:
            sim_dir = parameters.path
            # read parameters
            parameters = analysis.DataReader.load_parameters(sim_dir)
            # get tstop
            tstop = parameters.h_tstop
            
            # get builder runtime; read parameters.path/build_runtime.txt
            build_runtime_path = f"{sim_dir}/builder_runtime.txt"
            with open(build_runtime_path, 'r') as file:
                build_runtime = file.read()

            # get simulation runtime; read parameters.path/simulation_runtime.txt
            simulation_runtime_path = f"{sim_dir}/simulation_runtime.txt"
            with open(simulation_runtime_path, 'r') as file:
                simulation_runtime = file.read()
                
            replacement_runtime_path = f"{sim_dir}/replace_runtime.txt"
            with open(replacement_runtime_path, 'r') as file:
                replacement_runtime = file.read()

            # Store the runtimes in a dictionary
            sim_name = sim_dir.split('/')[-1]  # Get the last part of the path as the simulation name
            runtimes[sim_name] = {
                'build_time': build_runtime,
                'simulation_time': simulation_runtime,
                'replacement_time': replacement_runtime
            }
            
        
    except:
        from scripts.compare_sta import get_all_directories_within
        dirs = get_all_directories_within(sim.path)#'05-06-2024-14-19-43-Full120Sec/')
        for sim_dir in dirs:
                        # read parameters
                parameters = analysis.DataReader.load_parameters(sim_dir)
                # get tstop
                tstop = parameters.h_tstop
                
                # get builder runtime; read parameters.path/build_runtime.txt
                build_runtime_path = f"{sim_dir}/builder_runtime.txt"
                with open(build_runtime_path, 'r') as file:
                    build_runtime = file.read()

                # get simulation runtime; read parameters.path/simulation_runtime.txt
                simulation_runtime_path = f"{sim_dir}/simulation_runtime.txt"
                with open(simulation_runtime_path, 'r') as file:
                    simulation_runtime = file.read()
                    
                replacement_runtime_path = f"{sim_dir}/replace_runtime.txt"
                with open(replacement_runtime_path, 'r') as file:
                    replacement_runtime = file.read()

                # Store the runtimes in a dictionary
                sim_name = sim_dir.split('/')[-1]  # Get the last part of the path as the simulation name
                runtimes[sim_name] = {
                    'build_time': build_runtime,
                    'simulation_time': simulation_runtime,
                    'replacement_time': replacement_runtime
                }

In [ ]:
if common_attributes_to_use == 'sta':

    # Group runtimes by simulation type
    grouped_runtimes = {}

    for sim_key, times in runtimes.items():
        sim_type = sim_key.split('_')[0]
        if sim_type not in grouped_runtimes:
            grouped_runtimes[sim_type] = {'build_times': [], 'simulation_times': [], 'replacement_times': []}
        
        grouped_runtimes[sim_type]['build_times'].append(float(times['build_time'].replace(' seconds', '')))
        grouped_runtimes[sim_type]['simulation_times'].append(float(times['simulation_time'].replace(' seconds', '')))
        grouped_runtimes[sim_type]['replacement_times'].append(float(times['replacement_time'].replace(' seconds', '')))

    # Compute the mean and std of runtimes for each simulation type
    summary_stats = {}

    for sim_type, times in grouped_runtimes.items():
        build_times = times['build_times']
        simulation_times = times['simulation_times']
        replacement_times = times['replacement_times']

        summary_stats[sim_type] = {
            'build_time_mean': np.mean(build_times),
            'build_time_std': np.std(build_times),
            'simulation_time_mean': np.mean(simulation_times),
            'simulation_time_std': np.std(simulation_times),
            'replacement_time_mean': np.mean(replacement_times),
            'replacement_time_std': np.std(replacement_times)
        }

    # Print the summary statistics
    for sim_type, stats in summary_stats.items():
        # print(f"Simulation type: {sim_type}")
        # print(f"  Build time mean: {stats['build_time_mean']:.2f} seconds")
        # print(f"  Build time std: {stats['build_time_std']:.2f} seconds")
        # print(f"  Simulation time mean: {stats['simulation_time_mean']:.2f} seconds")
        # print(f"  Simulation time std: {stats['simulation_time_std']:.2f} seconds")
        # print(f"  Replacement time mean: {stats['replacement_time_mean']:.2f} seconds")
        # print(f"  Replacement time std: {stats['replacement_time_std']:.2f} seconds")
        
        # print(f"                Total time mean : {sum([stats['replacement_time_mean'], stats['simulation_time_mean'], stats['build_time_mean']]):.1f} seconds")
        print(f"{sim_type} : {sum([stats['replacement_time_mean'], stats['simulation_time_mean'], stats['build_time_mean']]):.1f} (s)")

## Check LFP

## TESTING

In [ ]:
print(sim.path)